In [3]:
import pandas as pd
import numpy as np
df = pd.read_excel(r'C:\Users\USER\Desktop\web-scraping-\final.xlsx')

In [4]:
import gensim
from gensim.models.ldamulticore import LdaMulticore
from gensim import corpora, models
import pyLDAvis.gensim_models
from nltk.corpus import stopwords
import string
from nltk.stem.wordnet import WordNetLemmatizer

import warnings
warnings.simplefilter('ignore')
from itertools import chain

In [5]:
#import nltk
#nltk.download('all')

In [6]:
df.head()

,topic,title,article,date,link
0,BUSINESS,"MAHB faces double whammy of interstate ban, Tu...",KUALA LUMPUR: Affin Hwang Capital expects Mala...,"May 4, 2021 @ 10:40am",https://www.nst.com.my/business/2021/05/687692...
1,BUSINESS,RM opens at 4.09 against USD,KUALA LUMPUR: The ringgit opened easier agains...,"May 3, 2021 @ 10:09am",https://www.nst.com.my/business/2021/05/687398...
2,COLUMNISTS,Biden needs fresh thinking and more than one term,FORGET about the first 100 days. US President ...,"May 3, 2021 @ 12:10am",https://www.nst.com.my/opinion/columnists/2021...
3,SUNDAY VIBES,MONEY THOUGHTS: Ikigai — The uncommon key to c...,HAVE you noticed how life's getting tougher on...,"May 2, 2021 @ 8:30am",https://www.nst.com.my/lifestyle/sunday-vibes/...
4,BUSINESS,"Labuan FSA appoints new chairman, director gen...",KUALA LUMPUR: Labuan Financial Services Author...,"Apr 30, 2021 @ 4:15pm",https://www.nst.com.my/business/2021/04/686731...


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   topic    107 non-null    object
 1   title    107 non-null    object
 2   article  107 non-null    object
 3   date     107 non-null    object
 4   link     107 non-null    object
dtypes: object(5)
memory usage: 4.3+ KB


In [8]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def clean(text):
    stop_free = ' '.join([word for word in text.lower().split() if word not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = ' '.join([lemma.lemmatize(word) for word in punc_free.split()])
    return normalized.split()

In [9]:
df['article_clean']=df['article'].apply(clean)

In [10]:
dictionary = corpora.Dictionary(df['article_clean'])
print(dictionary.num_nnz)

20580


In [11]:
doc_term_matrix = [dictionary.doc2bow(doc) for doc in df['article_clean'] ]
print(len(doc_term_matrix))

107


In [12]:
lda = gensim.models.ldamodel.LdaModel

In [13]:
num_topics= 8
%time ldamodel = lda(doc_term_matrix,num_topics=num_topics,id2word=dictionary,passes=10,minimum_probability=0)

Wall time: 1.95 s


In [14]:
ldamodel.print_topics(num_topics=num_topics)

[(0,
  '0.009*"per" + 0.009*"said" + 0.008*"cent" + 0.006*"malaysia" + 0.005*"also" + 0.004*"country" + 0.004*"u" + 0.004*"rating" + 0.004*"year" + 0.004*"asset"'),
 (1,
  '0.013*"said" + 0.008*"per" + 0.007*"cent" + 0.007*"year" + 0.006*"malaysia" + 0.006*"also" + 0.005*"million" + 0.005*"new" + 0.005*"government" + 0.004*"budget"'),
 (2,
  '0.020*"said" + 0.013*"per" + 0.012*"cent" + 0.010*"malaysia" + 0.008*"trade" + 0.007*"bhd" + 0.007*"investment" + 0.006*"market" + 0.006*"would" + 0.006*"u"'),
 (3,
  '0.000*"said" + 0.000*"cent" + 0.000*"u" + 0.000*"per" + 0.000*"would" + 0.000*"2020" + 0.000*"trade" + 0.000*"market" + 0.000*"also" + 0.000*"government"'),
 (4,
  '0.011*"bank" + 0.011*"said" + 0.007*"policy" + 0.007*"market" + 0.006*"price" + 0.006*"rate" + 0.005*"the" + 0.005*"dollar" + 0.005*"u" + 0.004*"government"'),
 (5,
  '0.010*"government" + 0.008*"country" + 0.006*"economic" + 0.006*"covid19" + 0.006*"people" + 0.006*"food" + 0.005*"malaysia" + 0.005*"also" + 0.005*"time"

In [15]:
lda_display = pyLDAvis.gensim_models.prepare(ldamodel, doc_term_matrix, dictionary, sort_topics=False, mds='mmds')
pyLDAvis.display(lda_display)

In [16]:
lda_corpus = ldamodel[doc_term_matrix]

In [30]:
[doc for doc in lda_corpus]

[[(0, 0.0005565066),
  (1, 0.0005563257),
  (2, 0.00055624516),
  (3, 0.0005564985),
  (4, 0.00055622257),
  (5, 0.0005563265),
  (6, 0.0005566399),
  (7, 0.99610525)],
 [(0, 0.0008876129),
  (1, 0.0008876936),
  (2, 0.0008872006),
  (3, 0.00088717096),
  (4, 0.9937878),
  (5, 0.0008869682),
  (6, 0.0008873248),
  (7, 0.0008881902)],
 [(0, 0.9990548),
  (1, 0.00013501251),
  (2, 0.00013500541),
  (3, 0.00013507104),
  (4, 0.00013504401),
  (5, 0.00013498246),
  (6, 0.0001350948),
  (7, 0.00013503793)],
 [(0, 0.00022558797),
  (1, 0.00022548425),
  (2, 0.0002254848),
  (3, 0.00022560514),
  (4, 0.00022551283),
  (5, 0.00022542948),
  (6, 0.99842143),
  (7, 0.00022551448)],
 [(0, 0.9937859),
  (1, 0.0008876246),
  (2, 0.00088776596),
  (3, 0.00088807486),
  (4, 0.0008876023),
  (5, 0.00088724477),
  (6, 0.00088784145),
  (7, 0.0008879394)],
 [(0, 0.0005959535),
  (1, 0.99582875),
  (2, 0.0005956321),
  (3, 0.0005956964),
  (4, 0.0005963388),
  (5, 0.00059558114),
  (6, 0.0005957531),
  (

In [16]:
scores = list(chain(*[[score for topic_id,score in topic] \
                      for topic in [doc for doc in lda_corpus]]))

threshold = sum(scores)/len(scores)
print(threshold)

0.11111111119094265


In [17]:
cluster1 = [j for i,j in zip(lda_corpus,df.index) if i[0][1] > threshold]
cluster2 = [j for i,j in zip(lda_corpus,df.index) if i[1][1] > threshold]
cluster3 = [j for i,j in zip(lda_corpus,df.index) if i[2][1] > threshold]
cluster4 = [j for i,j in zip(lda_corpus,df.index) if i[3][1] > threshold]
cluster5 = [j for i,j in zip(lda_corpus,df.index) if i[4][1] > threshold]

In [18]:
df.iloc[cluster1]

,topic,title,article,date,link,article_clean
22,BUSINESS,"Export conversion rule, other liberalisation m...",KUALA LUMPUR: Bank Negara Malaysia has announc...,"Mar 31, 2021 @ 10:15am",https://www.nst.com.my/business/2021/03/678354...,"[kuala, lumpur, bank, negara, malaysia, announ..."
53,BOTS,"#TECH: Nyan Cat GIF fetched nearly US$500,000 ...",WHO would have thought that an animated flying...,"Feb 24, 2021 @ 8:49pm",https://www.nst.com.my/lifestyle/bots/2021/02/...,"[would, thought, animated, flying, cat, leavin..."
63,BUSINESS,"Musk says he's a Bitcoin 'supporter,' buffetin...",Elon Musk once again sent ripples through the ...,"Feb 1, 2021 @ 5:16pm",https://www.nst.com.my/business/2021/02/662164...,"[elon, musk, sent, ripple, price, bitcoin, dec..."
64,SUNDAY VIBES,There's no stopping this force of nature as he...,"WHEN I was 15, I auditioned to sing for a char...","Jan 31, 2021 @ 8:46am",https://www.nst.com.my/lifestyle/sunday-vibes/...,"[15, auditioned, sing, charity, concert, can, ..."
66,GOVERNMENT / PUBLIC POLICY,Tengku Zafrul: No total lockdown even in worst...,KUALA LUMPUR: Finance Minister Tengku Datuk Se...,"Jan 29, 2021 @ 2:51pm",https://www.nst.com.my/news/government-public-...,"[kuala, lumpur, finance, minister, tengku, dat..."
70,NATION,Seremban collector impresses with Coca-Cola co...,SEREMBAN: What started as a hobby in collectin...,"Jan 24, 2021 @ 5:02pm",https://www.nst.com.my/news/nation/2021/01/660...,"[seremban, started, hobby, collecting, cocacol..."
96,WORLD,Yemen staring at 'world's worst famine',"UNITED NATIONS, United States: War-torn Yemen ...","Nov 21, 2020 @ 3:29pm",https://www.nst.com.my/world/world/2020/11/643...,"[united, nation, united, state, wartorn, yemen..."


In [32]:
df.iloc[cluster5]

,topic,title,article,date,link,article_clean
2,COLUMNISTS,Biden needs fresh thinking and more than one term,FORGET about the first 100 days. US President ...,"May 3, 2021 @ 12:10am",https://www.nst.com.my/opinion/columnists/2021...,"[forget, first, 100, day, u, president, joe, b..."
18,BUSINESS,Malaysia bond laggards show signs of recovery,MALAYSIAN corporate dollar bonds are the worst...,"Apr 14, 2021 @ 4:05pm",https://www.nst.com.my/business/2021/04/682290...,"[malaysian, corporate, dollar, bond, worst, pe..."
23,CRIME & COURTS,"'Nicky Gang' busted, Datuk Seri Nicky still mi...",KUALA LUMPUR: The police have arrested 68 indi...,"Mar 30, 2021 @ 7:21pm",https://www.nst.com.my/news/crime-courts/2021/...,"[kuala, lumpur, police, arrested, 68, individu..."
51,BUSINESS,Ringgit maintains 4.04 level against US dollar...,KUALA LUMPUR: After closing marginally higher ...,"Feb 25, 2021 @ 10:09am",https://www.nst.com.my/business/2021/02/668784...,"[kuala, lumpur, closing, marginally, higher, w..."
54,BUSINESS,Ageson's Q2 net profit increases 39pct to RM12...,KUALA LUMPUR: Ageson Bhd posted a net profit o...,"Feb 24, 2021 @ 6:31pm",https://www.nst.com.my/business/2021/02/668668...,"[kuala, lumpur, ageson, bhd, posted, net, prof..."
55,CRIME & COURTS,RM2.8 million lost to investment fraud syndica...,KUALA LUMPUR: Police arrested nine men on susp...,"Feb 6, 2021 @ 9:01am",https://www.nst.com.my/news/crime-courts/2021/...,"[kuala, lumpur, police, arrested, nine, men, s..."
64,SUNDAY VIBES,There's no stopping this force of nature as he...,"WHEN I was 15, I auditioned to sing for a char...","Jan 31, 2021 @ 8:46am",https://www.nst.com.my/lifestyle/sunday-vibes/...,"[15, auditioned, sing, charity, concert, can, ..."
68,BUSINESS,Ringgit strengthens against all major currencies,KUALA LUMPUR: The ringgit reversed recent loss...,"Jan 26, 2021 @ 10:10am",https://www.nst.com.my/business/2021/01/660574...,"[kuala, lumpur, ringgit, reversed, recent, los..."
80,BUSINESS,Tumultuous 2020 rewards investors who stared d...,THE bell has just tolled on one of the wildest...,"Jan 1, 2021 @ 3:14pm",https://www.nst.com.my/business/2021/01/653730...,"[bell, tolled, one, wildest, year, wall, stree..."
83,BUSINESS,Covid-19 uncertainties to ease with 2021 appro...,KUALA LUMPUR: Asia Pacific Investment Bank (AP...,"Dec 30, 2020 @ 11:30am",https://www.nst.com.my/business/2020/12/653140...,"[kuala, lumpur, asia, pacific, investment, ban..."
